# Tuning with K fold cross validation

In [ ]:
with_storage = False

In [ ]:
import optuna

In [ ]:
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.svm import OneClassSVM
from sklearn.neighbors import KDTree, BallTree
import pandas as pd
from sklearn.metrics import (
    silhouette_score,
    davies_bouldin_score,
    calinski_harabasz_score,
)


class DBOCSVM:
    def __init__(
        self,
        kernel="rbf",
        degree=3,
        gamma="scale",
        coef0=0.0,
        tol=0.001,
        nu=0.5,
        shrinking=True,
        cache_size=200,
        verbose=False,
        max_iter=-1,
        eps=0.5,
        min_samples=10,
        tree_metric="euclidean",
        dbscan_metric="euclidean",
        algorithm="kd_tree",  # or 'ball_tree'
        leaf_size=30,
        n_jobs=-1,  # Add n_jobs parameter
    ):
        self.kernel = kernel
        self.degree = degree
        self.gamma = gamma
        self.coef0 = coef0
        self.tol = tol
        self.nu = nu
        self.shrinking = shrinking
        self.cache_size = cache_size
        self.verbose = verbose
        self.max_iter = max_iter
        self.algorithm = algorithm
        self.leaf_size = leaf_size
        self.tree_metric = tree_metric
        self.dbscan_metric = dbscan_metric
        self.dbscan = DBSCAN(
            eps=eps, min_samples=min_samples, n_jobs=n_jobs, metric=dbscan_metric
        )  # Make it so that it can accept a metric parameter
        self.svms = {}  # One SVM per cluster
        self.dbscan_centroids = {}  # To store cluster centroids
        self.cluster_points = {}  # Store points in each cluster
        self.tree = None
        # These attributes are mainly used for inspection purposes
        self.cluster_sizes = {}  # Number of points in each cluster
        self.n_jobs = n_jobs  # Store n_jobs
        self.cluster_labels = None
        self.unique_clusters = None

    def fit_cluster(
        self,
        X,
        dbscan_evaluation_metric="silhouette",
        dbscan_rerun=False,
        dbscan_rerun_trials=10,
        verbose=False,
    ):
        """
        Parameters:
        -----------
        X : array-like
            Training data
        dbscan_evaluation_metric : str
            Metric to optimize ('silhouette', 'davies_bouldin', or 'calinski_harabasz')
        dbscan_rerun : bool
            Whether to rerun DBSCAN after fitting the model with the best parameters
        dbscan_rerun_trials : int
            Number of reruns for DBSCAN after fitting the model with the best parameters
        """

        X = X.values if isinstance(X, pd.DataFrame) else X

        """
        NOTE: Current DBSCAN only uses euclidean distance, so the metric parameter is not used
        TODO: Add metric parameter to DBSCAN to handle different distance metrics
        'euclidean': Standard Euclidean distance. This is the default metric.
        'manhattan': Manhattan or L1 distance (sum of absolute differences).
        'chebyshev': Chebyshev or maximum distance.
        'minkowski': Minkowski distance, a generalization of Euclidean and Manhattan distance. The power parameter p of the Minkowski metric can be controlled by the p parameter of DBSCAN.
        'wminkowski': Weighted Minkowski distance.
        'seuclidean': Standardized Euclidean distance.
        'mahalanobis': Mahalanobis distance.
        """
        if verbose:
            print("Fitting DBSCAN...")
        # NOTE: we use the dbscan that was initialized in the constructor
        self.cluster_labels = self.dbscan.fit_predict(X)
        if verbose:
            print("DBSCAN Fitted...")

        if dbscan_rerun:
            if verbose:
                print("Rerunning DBSCAN...")

            if dbscan_evaluation_metric == "silhouette":
                current_score = silhouette_score(X, self.cluster_labels)
            elif dbscan_evaluation_metric == "davies_bouldin":
                current_score = davies_bouldin_score(X, self.cluster_labels)
            else:  # calinski_harabasz
                current_score = calinski_harabasz_score(X, self.cluster_labels)

            for i in range(dbscan_rerun_trials):
                if verbose:
                    print(f"DBSCAN Rerun {i+1}...")
                new_cluster_labels = self.dbscan.fit_predict(X)

                if dbscan_evaluation_metric == "silhouette":
                    new_score = silhouette_score(X, new_cluster_labels)
                    if new_score > current_score:
                        self.cluster_labels = new_cluster_labels
                        current_score = new_score
                elif dbscan_evaluation_metric == "davies_bouldin":
                    new_score = davies_bouldin_score(X, new_cluster_labels)
                    if new_score < current_score:
                        self.cluster_labels = new_cluster_labels
                        current_score = new_score
                else:  # calinski_harabasz
                    new_score = calinski_harabasz_score(X, new_cluster_labels)
                    if new_score > current_score:
                        self.cluster_labels = new_cluster_labels
                        current_score = new_score

        self.unique_clusters = np.unique(self.cluster_labels)

        if verbose:
            print(f"Unique Clusters: {self.unique_clusters}")

        for cluster in self.unique_clusters:
            # Store the number of points in the cluster
            # mainly for inspection purposes
            n_points = np.sum(self.cluster_labels == cluster)
            self.cluster_sizes[int(cluster)] = int(n_points)

        if verbose:
            print(f"Cluster Sizes: {self.cluster_sizes}")

    def fit_ocsvm(
        self,
        X,
        parameter_list=None,
        verbose=False,
    ):
        """
        Parameters:
        -----------
        X : array-like
            Training data
        parameter_list: dictionary of dictionaries
            Each key in the dictionary is the cluster number and
            the value is a dictionary containing the parameters for OCSVM
            each dictionary looks like this:
            {
                0 : {
                kernel: rbf, linear, poly, or sigmoid,
                gamma: 'scale', 'auto' or a float,
                nu: a float between 0 and 1 e.g 0.2,
                }
            }
        """
        X = X.values if isinstance(X, pd.DataFrame) else X

        if parameter_list is None:
            raise ValueError("parameter_list cannot be None")

        if len(parameter_list) < len(self.unique_clusters) - 1:
            raise ValueError(
                "Number of parameters should be equal or greater than the number of clusters"
            )

        def filter_dict(original_dict, keys_to_keep):
            return {k: original_dict[k] for k in keys_to_keep if k in original_dict}

        if len(parameter_list) >= len(self.unique_clusters) - 1:
            cluster_count = list(self.cluster_sizes.keys())
            cluster_count.remove(-1)
            cluster_count

            parameter_list = filter_dict(parameter_list, cluster_count)

        for cluster in self.unique_clusters:

            if cluster == -1:  # Skip noise cluster for SVM training
                continue

            if verbose:
                print(
                    f"Training for cluster {cluster} with {self.cluster_sizes[cluster]} points"
                )

            # Boolean masking to get points in the current cluster
            points = X[self.cluster_labels == cluster]
            self.cluster_points[cluster] = points

            if len(points) > 0:
                # use parameters defined in constructor if not provided
                if parameter_list is None:
                    ocsvm = OneClassSVM(
                        kernel=self.kernel,
                        nu=self.nu,
                        gamma=self.gamma,
                        degree=self.degree,
                        coef0=self.coef0,
                        tol=self.tol,
                        shrinking=self.shrinking,
                        cache_size=self.cache_size,
                        max_iter=self.max_iter,
                    )
                else:
                    ocsvm = OneClassSVM(
                        kernel=parameter_list[cluster]["kernel"],
                        nu=parameter_list[cluster]["nu"],
                        gamma=parameter_list[cluster]["gamma"],
                        degree=self.degree,
                        coef0=self.coef0,
                        tol=self.tol,
                        shrinking=self.shrinking,
                        cache_size=self.cache_size,
                        max_iter=self.max_iter,
                    )
                    if verbose:
                        print(
                            f"OCSVM for cluster {cluster} uses nu: {parameter_list[cluster]['nu']}, gamma: {parameter_list[cluster]['gamma']}, kernel: {parameter_list[cluster]['kernel']}"
                        )
                ocsvm.fit(points)

                self.svms[cluster] = ocsvm

                """
                TODO: Explore other alternatives for centroid calculation
                "->" means the following line might be a downside of the current approach.
                
                - Median: More robust to outliers than the mean (`np.median(points, axis=0)`).
                    -> Less representative if data is asymmetric  
                - Trimmed Mean: Removes extreme values before computing the mean (`scipy.stats.trim_mean`).
                    ->   Requires choosing the trimming percentage
                - Weighted Mean: Assigns importance to points based on reliability.  
                    ->  Requires defining weights
                - Geometric Median: Minimizes sum of distances to all points. More robust to outliers than the mean.
                    -> computationally expensive (`scipy.spatial`)
                - Distance Metrics: Use median for Manhattan distance and mean for Euclidean distance.
                    -> Requires choosing the distance metric
                    
                """
                self.dbscan_centroids[cluster] = np.mean(points, axis=0)

        # Build tree with cluster centroids
        centroids = [self.dbscan_centroids[c] for c in self.dbscan_centroids if c != -1]
        self.valid_clusters = list(self.dbscan_centroids.keys())
        if len(centroids) > 0:
            centroids = np.array(centroids)
            if self.algorithm == "kd_tree":
                self.tree = KDTree(
                    centroids, leaf_size=self.leaf_size, metric=self.tree_metric
                )
            elif self.algorithm == "ball_tree":
                self.tree = BallTree(
                    centroids, leaf_size=self.leaf_size, metric=self.tree_metric
                )

    def fit(
        self,
        X,
        dbscan_evaluation_metric="silhouette",  # only used for reruns
        dbscan_rerun=False,  # only used for reruns
        dbscan_rerun_trials=10,  # only used for reruns
        parameter_list=None,
        verbose=False,
    ):
        """
        Parameters:
        -----------
        X : array-like
            Training data
        dbscan_evaluation_metric : str
            Metric to optimize ('silhouette', 'davies_bouldin', or 'calinski_harabasz')
        dbscan_rerun : bool
            Whether to rerun DBSCAN after fitting the model with the best parameters
        dbscan_rerun_trials : int
            Number of reruns for DBSCAN after fitting the model with the best parameters
        parameter_list: dictionary of dictionaries
            Each key in the dictionary is the cluster number and
            the value is a dictionary containing the parameters for OCSVM
            each dictionary looks like this:
            {
                0 : {
                kernel: rbf, linear, poly, or sigmoid,
                gamma: 'scale', 'auto' or a float,
                nu: a float between 0 and 1 e.g 0.2,
                }
            }
        """

        X = X.values if isinstance(X, pd.DataFrame) else X

        """
        NOTE: Current DBSCAN only uses euclidean distance, so the metric parameter is not used
        TODO: Add metric parameter to DBSCAN to handle different distance metrics
        'euclidean': Standard Euclidean distance. This is the default metric.
        'manhattan': Manhattan or L1 distance (sum of absolute differences).
        'chebyshev': Chebyshev or maximum distance.
        'minkowski': Minkowski distance, a generalization of Euclidean and Manhattan distance. The power parameter p of the Minkowski metric can be controlled by the p parameter of DBSCAN.
        'wminkowski': Weighted Minkowski distance.
        'seuclidean': Standardized Euclidean distance.
        'mahalanobis': Mahalanobis distance.
        """
        if verbose:
            print("Fitting DBSCAN...")
        # NOTE: we use the dbscan that was initialized in the constructor
        cluster_labels = self.dbscan.fit_predict(X)
        if verbose:
            print("DBSCAN Fitted...")

        if dbscan_rerun:
            if verbose:
                print("Rerunning DBSCAN...")

            if dbscan_evaluation_metric == "silhouette":
                current_score = silhouette_score(X, cluster_labels)
            elif dbscan_evaluation_metric == "davies_bouldin":
                current_score = davies_bouldin_score(X, cluster_labels)
            else:  # calinski_harabasz
                current_score = calinski_harabasz_score(X, cluster_labels)

            for i in range(dbscan_rerun_trials):
                if verbose:
                    print(f"DBSCAN Rerun {i+1}...")

                new_cluster_labels = self.dbscan.fit_predict(X)

                if dbscan_evaluation_metric == "silhouette":
                    new_score = silhouette_score(X, new_cluster_labels)
                    if new_score > current_score:
                        cluster_labels = new_cluster_labels
                        current_score = new_score
                elif dbscan_evaluation_metric == "davies_bouldin":
                    new_score = davies_bouldin_score(X, new_cluster_labels)
                    if new_score < current_score:
                        cluster_labels = new_cluster_labels
                        current_score = new_score
                else:  # calinski_harabasz
                    new_score = calinski_harabasz_score(X, new_cluster_labels)
                    if new_score > current_score:
                        cluster_labels = new_cluster_labels
                        current_score = new_score

        unique_clusters = np.unique(cluster_labels)

        if verbose:
            print(f"Unique Clusters: {unique_clusters}")

        for cluster in unique_clusters:
            # Store the number of points in the cluster
            # mainly for inspection purposes
            n_points = np.sum(cluster_labels == cluster)
            self.cluster_sizes[int(cluster)] = int(n_points)

        if verbose:
            print(f"Cluster Sizes: {self.cluster_sizes}")

        if parameter_list is not None and (len(parameter_list)) < (
            len(unique_clusters) - 1
        ):
            raise ValueError(
                "Number of parameters should be equal or greater than the number of clusters"
            )

        def filter_dict(original_dict, keys_to_keep):
            return {k: original_dict[k] for k in keys_to_keep if k in original_dict}

        if parameter_list is not None and (len(parameter_list)) >= (
            len(unique_clusters) - 1
        ):
            cluster_count = list(self.cluster_sizes.keys())
            cluster_count.remove(-1)
            cluster_count

            parameter_list = filter_dict(parameter_list, cluster_count)

        self.parameter_list = parameter_list

        for cluster in unique_clusters:

            # Store the number of points in the cluster
            # n_points = np.sum(cluster_labels == cluster)
            # self.cluster_sizes[int(cluster)] = int(n_points)

            if cluster == -1:  # Skip noise cluster for SVM training
                continue

            if verbose:
                print(
                    f"Training for cluster {cluster} with {self.cluster_sizes[cluster]} points"
                )

            # Boolean masking to get points in the current cluster
            points = X[cluster_labels == cluster]
            self.cluster_points[cluster] = points

            if len(points) > 0:
                # use parameters defined in constructor if not provided
                if parameter_list is None:
                    ocsvm = OneClassSVM(
                        kernel=self.kernel,
                        nu=self.nu,
                        gamma=self.gamma,
                        degree=self.degree,
                        coef0=self.coef0,
                        tol=self.tol,
                        shrinking=self.shrinking,
                        cache_size=self.cache_size,
                        max_iter=self.max_iter,
                    )
                else:
                    ocsvm = OneClassSVM(
                        kernel=parameter_list[cluster]["kernel"],
                        nu=parameter_list[cluster]["nu"],
                        gamma=parameter_list[cluster]["gamma"],
                        degree=self.degree,
                        coef0=self.coef0,
                        tol=self.tol,
                        shrinking=self.shrinking,
                        cache_size=self.cache_size,
                        max_iter=self.max_iter,
                    )
                    if verbose:
                        print(
                            f"OCSVM for cluster {cluster} uses nu: {parameter_list[cluster]['nu']}, gamma: {parameter_list[cluster]['gamma']}, kernel: {parameter_list[cluster]['kernel']}"
                        )
                ocsvm.fit(points)

                self.svms[cluster] = ocsvm

                """
                TODO: Explore other alternatives for centroid calculation
                "->" means the following line might be a downside of the current approach.
                
                - Median: More robust to outliers than the mean (`np.median(points, axis=0)`).
                    -> Less representative if data is asymmetric  
                - Trimmed Mean: Removes extreme values before computing the mean (`scipy.stats.trim_mean`).
                    ->   Requires choosing the trimming percentage
                - Weighted Mean: Assigns importance to points based on reliability.  
                    ->  Requires defining weights
                - Geometric Median: Minimizes sum of distances to all points. More robust to outliers than the mean.
                    -> computationally expensive (`scipy.spatial`)
                - Distance Metrics: Use median for Manhattan distance and mean for Euclidean distance.
                    -> Requires choosing the distance metric
                    
                """
                self.dbscan_centroids[cluster] = np.mean(points, axis=0)

        # Build tree with cluster centroids
        centroids = [self.dbscan_centroids[c] for c in self.dbscan_centroids if c != -1]
        self.valid_clusters = list(self.dbscan_centroids.keys())
        if len(centroids) > 0:
            centroids = np.array(centroids)
            if self.algorithm == "kd_tree":
                self.tree = KDTree(
                    centroids, leaf_size=self.leaf_size, metric=self.tree_metric
                )
            elif self.algorithm == "ball_tree":
                self.tree = BallTree(
                    centroids, leaf_size=self.leaf_size, metric=self.tree_metric
                )

    def predict(self, X):
        predictions = np.ones(len(X))
        X = X.values if isinstance(X, pd.DataFrame) else X

        if self.tree is None:
            return -1 * np.ones(len(X))

        # Find nearest centroid
        dist, ind = self.tree.query(X, k=1)
        nearest_clusters = [self.valid_clusters[i] for i in ind.flatten()]

        for i, cluster in enumerate(nearest_clusters):
            if cluster in self.svms:
                predictions[i] = self.svms[cluster].predict([X[i]])[0]
            else:
                predictions[i] = -1  # Anomaly if no SVM for cluster

        return predictions


"""
Classification Report on NSL-KDD:
              precision    recall  f1-score   support

     Anomaly       0.57      1.00      0.73     12832
      Normal       1.00      0.00      0.00      9711

    accuracy                           0.57     22543
   macro avg       0.78      0.50      0.36     22543
weighted avg       0.75      0.57      0.41     22543

Precision: 0.5693495429940545
Recall: 1.0
F1 Score: 0.7255866553576478
"""

"""
Tuning on the validation set results in extreme overfitting
Model performs better but only on the validation set, the test set results are worse
Cannot use the full train set since it crashed the kernel
TODO: Implement a better tuning strategy that generalizes well
TODO: Explore other preprocessing steps e.g Autoencoder, Min-Max Normalization, Feature Selection
"""

# Tuning

In [ ]:
import pandas as pd

### Train set

In [ ]:
train_set_path = (
    "/home/jbct/Projects/thesis/db-ocsvm/data/processed/NSL-KDD/train_set_unsplit.csv"
)
train_set = pd.read_csv(train_set_path)
train_set = train_set.sample(frac=0.1, random_state=42).reset_index(drop=True)
print(train_set.shape)
train_set.head(10)

In [ ]:
X = train_set.drop(["attack_binary", "attack_categorical", "attack_class"], axis=1)
y = train_set["attack_binary"]

In [ ]:
X.head(3)

In [ ]:
y.head(3)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA


def objective(trial):
    parameter_list = {}
    for cluster in range(0, 20):
        hyperparameter = {
            "kernel": "rbf",
            "gamma": trial.suggest_float(f"gamma_{cluster}", 1e-4, 1.0, log=True),
            "nu": trial.suggest_float(f"nu_{cluster}", 0.01, 0.5),
        }
        parameter_list[cluster] = hyperparameter

    kf = KFold(n_splits=4, shuffle=True, random_state=42)

    # Store scores for each fold
    scores = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        X_train_normal = X_train[y_train == 1]

        # if len(X_train_normal) > 31500:
        #     X_train_normal = X_train_normal.sample(n=32000, random_state=42)

        pca = PCA(n_components=69)
        X_normal_pca = pca.fit_transform(X_train_normal)

        X_train_normal = pd.DataFrame(
            data=X_normal_pca,
            columns=[f"PC{i+1}" for i in range(pca.n_components_)],
            index=X_train_normal.index,  # This preserves the original index
        )

        dbocsvm = DBOCSVM(
            kernel="rbf",
            gamma="auto",
            nu=0.2,
            eps=8,
            min_samples=5,
            algorithm="ball_tree",
        )

        dbocsvm.fit(X_train_normal.values, parameter_list=parameter_list)

        X_val_pca = pca.transform(X_val)
        X_val = pd.DataFrame(
            data=X_val_pca,
            columns=[f"PC{i+1}" for i in range(pca.n_components_)],
            index=X_val.index,  # This preserves the original index
        )

        y_pred = dbocsvm.predict(X_val.values)

        # Calculate F1 score for this fold
        score = f1_score(y_val, y_pred, pos_label=-1)
        scores.append(score)

    # Return mean F1 score across all folds
    return np.mean(scores)

In [ ]:
storage_path = "sqlite:///optuna_storage/study_proposed_03.db"
trials = 7

if with_storage:
    study = optuna.create_study(
        direction="maximize",
        storage=storage_path,
        study_name="study_proposed_03",
        load_if_exists=True,
    )
    study.optimize(objective, n_trials=trials)
else:
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=trials)

In [ ]:
print(f"Best f1 score: {study.best_value:.3f}")
print(f"Best parameters: {study.best_params}")

In [ ]:
parameter_list = {}

for key, value in study.best_params.items():
    cluster = key.split("_")[1]
    cluster = int(cluster)

    parameter_list[cluster] = {
        "kernel": "rbf",
        "gamma": study.best_params[f"gamma_{cluster}"],
        "nu": study.best_params[f"nu_{cluster}"],
    }

# Training with full dataset

In [ ]:
train_set_full_path = (
    "/home/jbct/Projects/thesis/db-ocsvm/data/processed/NSL-KDD/train_set_full.csv"
)
train_dataset_full = pd.read_csv(train_set_full_path)
train_dataset_full = train_dataset_full.sample(frac=0.14, random_state=42).reset_index(
    drop=True
)
print(train_dataset_full.shape)
train_dataset_full.head(3)

In [ ]:
dbocsvm = DBOCSVM(
    kernel="rbf",
    gamma="auto",
    nu=0.2,
    eps=8,
    min_samples=5,
    algorithm="ball_tree",
)
print(dbocsvm.n_jobs)
dbocsvm.fit(train_dataset_full.values, parameter_list=parameter_list, verbose=True)

# Testing

Preparing test set

In [ ]:
test_set_path = (
    "/home/jbct/Projects/thesis/db-ocsvm/data/processed/NSL-KDD/test_set.csv"
)

test_set = pd.read_csv(test_set_path)
print(f"test set count: {test_set.shape[0]:,}")

In [ ]:
test_set.head(3)

In [ ]:
X_test = test_set.drop(columns=["attack_binary", "attack_categorical", "attack_class"])

y_test = test_set["attack_binary"]
y_test_class = test_set["attack_class"]
print(y_test.unique())

Perform prediction

In [ ]:
y_pred = dbocsvm.predict(X_test.values)

In [ ]:
print(*y_pred[:10], sep="\n")

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred, labels=[-1, 1])

# Print the confusion matrix
print("Confusion Matrix:")
print(cm)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


def plot_confusion_matrix(cm, labels, title):
    plt.figure(figsize=(5, 4))
    sns.heatmap(
        cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels
    )
    plt.xlabel("Predicted Label")
    plt.ylabel("Actual Label")
    plt.title(title)
    plt.show()


plot_confusion_matrix(cm, ["Anomaly", "Normal"], "Confusion Matrix (Anomaly vs Normal)")

In [ ]:
from sklearn.metrics import (
    classification_report,
    precision_score,
    recall_score,
    f1_score,
    accuracy_score,
)

print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=["Anomaly", "Normal"]))

precision = precision_score(y_test, y_pred, pos_label=-1)
recall = recall_score(y_test, y_pred, pos_label=-1)
f1 = f1_score(y_test, y_pred, pos_label=-1)
acc = accuracy_score(y_test, y_pred)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {acc}")

In [ ]:
import json
import datetime

log_file = "/home/jbct/Projects/thesis/db-ocsvm/test.log.pca.json"

if acc > 0.84:
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    performance = {
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "accuracy": acc,
    }
    data_to_save = {
        "timestamp": timestamp,
        "parameters": dbocsvm.parameter_list,
        "performance": performance,
    }

    try:
        with open(log_file, "r") as f:
            existing_data = json.load(f)

        existing_data.append(data_to_save)
    except json.JSONDecodeError:
        existing_data = [data_to_save]

    with open(log_file, "w") as f:
        json.dump(existing_data, f, indent=4)

    print("Paramters saved to log file")
else:
    print("Parameters not saved to log file")
    print("Acc is below 0.85")

In [ ]:
def create_multiclass_cm(y_true_class, y_pred_binary):
    """
    Create a confusion matrix showing how each attack class was classified.

    For attack classes (DoS, Probe, R2L, U2R), correct detection is when y_pred = -1 (anomaly)
    For normal class, correct detection is when y_pred = 1 (normal)
    """
    classes = np.unique(y_true_class)
    cm = np.zeros((len(classes), 2))

    for i, cls in enumerate(classes):
        # Get predictions for this class
        cls_indices = y_true_class == cls
        preds = y_pred_binary[cls_indices]

        # Count correct and incorrect predictions
        if cls == "normal":
            cm[i, 0] = np.sum(preds == -1)  # incorrectly detected as anomaly
            cm[i, 1] = np.sum(preds == 1)  # correctly detected as normal
        else:
            cm[i, 0] = np.sum(preds == -1)  # correctly detected as anomaly
            cm[i, 1] = np.sum(preds == 1)  # incorrectly detected as normal

    return cm, classes


# Create and plot the multi-class confusion matrix
cm_multi, classes = create_multiclass_cm(y_test_class, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(
    cm_multi,
    annot=True,
    fmt="g",
    cmap="Blues",
    xticklabels=["Detected as Anomaly", "Detected as Normal"],
    yticklabels=classes,
)
plt.ylabel("True Attack Class")
plt.title("Confusion Matrix by Attack Class")
plt.tight_layout()
plt.show()

In [ ]:
# Calculate detection rates for each class
print("Detection rates by class:")
class_metrics = {}
for cls in np.unique(y_test_class):
    # Get indices for this class
    class_indices = y_test_class == cls

    # True values and predictions for this class
    y_true_cls = y_test[class_indices]
    y_pred_cls = y_pred[class_indices]

    # Calculate metrics
    if cls == "Normal":
        # For normal class, we want to detect 1 (normal)
        correct = np.sum((y_pred_cls == 1))
        precision = precision_score(
            y_true_cls, y_pred_cls, pos_label=1, zero_division=0
        )
        recall = recall_score(y_true_cls, y_pred_cls, pos_label=1, zero_division=0)
    else:
        # For attack classes, we want to detect -1 (anomaly)
        correct = np.sum((y_pred_cls == -1))
        precision = precision_score(
            y_true_cls, y_pred_cls, pos_label=-1, zero_division=0
        )
        recall = recall_score(y_true_cls, y_pred_cls, pos_label=-1, zero_division=0)

    total = len(y_pred_cls)
    detection_rate = correct / total
    f1 = f1_score(
        y_true_cls, y_pred_cls, pos_label=-1 if cls != "Normal" else 1, zero_division=0
    )

    class_metrics[cls] = {
        "detection_rate": detection_rate,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "count": total,
        "correctly_detected": correct,
    }

    print(f"{cls}: {detection_rate:.4f} ({correct}/{total})")